# 新建一个我们开发要用到数据库

In [3]:
import sqlite3
from datetime import date, datetime
import random
from typing import Optional

class StudentDatabase:
    def __init__(self, db_path: str = "students.db"):
        """初始化数据库连接"""
        self.db_path = db_path
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()
        self.create_tables()
    
    def create_tables(self):
        """创建学生基本信息表"""
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS students (
                student_id TEXT PRIMARY KEY,
                name TEXT NOT NULL,
                gender TEXT NOT NULL CHECK(gender IN ('男', '女')),
                birth_date DATE NOT NULL,
                zodiac_sign TEXT NOT NULL,
                mbti TEXT CHECK(length(mbti) = 4),
                class_id TEXT NOT NULL,
                enrollment_date DATE NOT NULL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        self.conn.commit()
    
    def calculate_zodiac_sign(self, birth_date: date) -> str:
        """根据出生日期计算星座"""
        month = birth_date.month
        day = birth_date.day
        
        zodiac_dates = [
            (1, 20, "摩羯座"), (2, 19, "水瓶座"), (3, 21, "双鱼座"),
            (4, 20, "白羊座"), (5, 21, "金牛座"), (6, 21, "双子座"),
            (7, 23, "巨蟹座"), (8, 23, "狮子座"), (9, 23, "处女座"),
            (10, 23, "天秤座"), (11, 22, "天蝎座"), (12, 22, "射手座"),
            (12, 31, "摩羯座")  # 年末的摩羯座
        ]
        
        for m, d, sign in zodiac_dates:
            if month < m or (month == m and day <= d):
                return sign
        return "摩羯座"
    
    def add_student(self, student_id: str, name: str, gender: str, 
                   birth_date: date, mbti: str, class_id: str, 
                   enrollment_date: Optional[date] = None):
        """添加学生信息"""
        if enrollment_date is None:
            enrollment_date = date.today()
        
        zodiac_sign = self.calculate_zodiac_sign(birth_date)
        
        try:
            self.cursor.execute('''
                INSERT INTO students 
                (student_id, name, gender, birth_date, zodiac_sign, mbti, class_id, enrollment_date)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ''', (student_id, name, gender, birth_date.isoformat(), 
                  zodiac_sign, mbti.upper(), class_id, enrollment_date.isoformat()))
            self.conn.commit()
            print(f"成功添加学生：{name} (ID: {student_id})")
        except sqlite3.IntegrityError as e:
            print(f"添加失败：{e}")
    
    def get_student(self, student_id: str):
        """查询单个学生信息"""
        self.cursor.execute('''
            SELECT * FROM students WHERE student_id = ?
        ''', (student_id,))
        
        student = self.cursor.fetchone()
        if student:
            columns = [desc[0] for desc in self.cursor.description]
            return dict(zip(columns, student))
        return None
    
    def get_all_students(self, class_id: Optional[str] = None):
        """查询所有学生或特定班级的学生"""
        if class_id:
            self.cursor.execute('''
                SELECT * FROM students WHERE class_id = ? ORDER BY student_id
            ''', (class_id,))
        else:
            self.cursor.execute('SELECT * FROM students ORDER BY class_id, student_id')
        
        students = self.cursor.fetchall()
        columns = [desc[0] for desc in self.cursor.description]
        return [dict(zip(columns, student)) for student in students]
    
    def update_student(self, student_id: str, **kwargs):
        """更新学生信息"""
        allowed_fields = ['name', 'gender', 'birth_date', 'mbti', 'class_id']
        update_fields = []
        values = []
        
        for field, value in kwargs.items():
            if field in allowed_fields:
                update_fields.append(f"{field} = ?")
                values.append(value)
        
        if update_fields:
            # 如果更新了生日，重新计算星座
            if 'birth_date' in kwargs:
                update_fields.append("zodiac_sign = ?")
                values.append(self.calculate_zodiac_sign(kwargs['birth_date']))
            
            values.append(student_id)
            query = f"UPDATE students SET {', '.join(update_fields)} WHERE student_id = ?"
            self.cursor.execute(query, values)
            self.conn.commit()
            print(f"学生 {student_id} 信息已更新")
    
    def delete_student(self, student_id: str):
        """删除学生信息"""
        self.cursor.execute('DELETE FROM students WHERE student_id = ?', (student_id,))
        self.conn.commit()
        print(f"学生 {student_id} 已删除")
    
    def close(self):
        """关闭数据库连接"""
        self.conn.close()


# 生成模拟数据的辅助函数
def generate_mock_students(db: StudentDatabase, class_id: str = "2024-C1", num_students: int = 30):
    """生成模拟的学生数据"""
    # 姓氏列表
    surnames = ['张', '王', '李', '赵', '刘', '陈', '杨', '黄', '周', '吴', 
                '徐', '孙', '马', '胡', '郭', '林', '何', '高', '罗', '郑']
    
    # 名字列表
    male_names = ['伟', '强', '磊', '洋', '勇', '杰', '涛', '明', '军', '波',
                  '辉', '鹏', '宇', '浩', '俊', '毅', '凯', '峰', '超', '腾']
    female_names = ['静', '丽', '娟', '芳', '敏', '燕', '玲', '霞', '婷', '慧',
                   '莹', '颖', '琳', '洁', '梅', '雪', '倩', '珍', '瑶', '露']
    
    # MBTI 类型
    mbti_types = ['INTJ', 'INTP', 'ENTJ', 'ENTP', 'INFJ', 'INFP', 'ENFJ', 'ENFP',
                  'ISTJ', 'ISFJ', 'ESTJ', 'ESFJ', 'ISTP', 'ISFP', 'ESTP', 'ESFP']
    
    # 生成学生数据
    for i in range(1, num_students + 1):
        student_id = f"2024{i:03d}"  # 生成学号：2024001, 2024002, ...
        
        # 随机性别
        gender = random.choice(['男', '女'])
        
        # 根据性别选择名字
        surname = random.choice(surnames)
        if gender == '男':
            name = surname + random.choice(male_names)
        else:
            name = surname + random.choice(female_names)
        
        # 随机生日（假设都是2008-2010年出生的）
        year = random.randint(2008, 2010)
        month = random.randint(1, 12)
        day = random.randint(1, 28)  # 简化处理，避免月份日期问题
        birth_date = date(year, month, day)
        
        # 随机MBTI
        mbti = random.choice(mbti_types)
        
        # 入学日期
        enrollment_date = date(2024, 9, 1)
        
        # 添加到数据库
        db.add_student(student_id, name, gender, birth_date, mbti, class_id, enrollment_date)


# 使用示例
if __name__ == "__main__":
    # 创建数据库实例
    db = StudentDatabase("school_management.db")
    
    # 生成30个学生的模拟数据
    print("开始生成模拟数据...")
    generate_mock_students(db, "2024-C1", 30)
    
    # 查询所有学生
    print("\n班级所有学生信息：")
    students = db.get_all_students("2024-C1")
    for student in students:
        print(f"学号: {student['student_id']}, 姓名: {student['name']}, "
              f"性别: {student['gender']}, 星座: {student['zodiac_sign']}, "
              f"MBTI: {student['mbti']}")
    
    # 查询单个学生
    print("\n查询单个学生示例：")
    student = db.get_student("2024001")
    if student:
        print(f"找到学生：{student}")
    
    # 关闭数据库
    db.close()

开始生成模拟数据...
成功添加学生：黄婷 (ID: 2024001)
成功添加学生：杨浩 (ID: 2024002)
成功添加学生：杨娟 (ID: 2024003)
成功添加学生：马腾 (ID: 2024004)
成功添加学生：马丽 (ID: 2024005)
成功添加学生：陈强 (ID: 2024006)
成功添加学生：马燕 (ID: 2024007)
成功添加学生：郭珍 (ID: 2024008)
成功添加学生：孙毅 (ID: 2024009)
成功添加学生：胡强 (ID: 2024010)
成功添加学生：林燕 (ID: 2024011)
成功添加学生：郑磊 (ID: 2024012)
成功添加学生：周峰 (ID: 2024013)
成功添加学生：徐珍 (ID: 2024014)
成功添加学生：孙颖 (ID: 2024015)
成功添加学生：马慧 (ID: 2024016)
成功添加学生：黄毅 (ID: 2024017)
成功添加学生：马超 (ID: 2024018)
成功添加学生：张娟 (ID: 2024019)
成功添加学生：马慧 (ID: 2024020)
成功添加学生：胡伟 (ID: 2024021)
成功添加学生：高婷 (ID: 2024022)
成功添加学生：张杰 (ID: 2024023)
成功添加学生：孙珍 (ID: 2024024)
成功添加学生：郭瑶 (ID: 2024025)
成功添加学生：赵强 (ID: 2024026)
成功添加学生：陈磊 (ID: 2024027)
成功添加学生：何瑶 (ID: 2024028)
成功添加学生：罗波 (ID: 2024029)
成功添加学生：张腾 (ID: 2024030)

班级所有学生信息：
学号: 2024001, 姓名: 黄婷, 性别: 女, 星座: 天蝎座, MBTI: INTJ
学号: 2024002, 姓名: 杨浩, 性别: 男, 星座: 狮子座, MBTI: ISTP
学号: 2024003, 姓名: 杨娟, 性别: 女, 星座: 巨蟹座, MBTI: ISTJ
学号: 2024004, 姓名: 马腾, 性别: 男, 星座: 天蝎座, MBTI: ENFP
学号: 2024005, 姓名: 马丽, 性别: 女, 星座: 金牛座, MBTI: ENFJ
学号: 2024006, 姓名: 

In [4]:
import sqlite3
import os
from datetime import datetime

class DatabaseSchemaCreator:
    def __init__(self, db_path: str = "school_management.db"):
        self.db_path = db_path
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()
        
    def create_teachers_table(self):
        """创建教师表"""
        print("创建 teachers 表...")
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS teachers (
                teacher_id TEXT PRIMARY KEY,
                name TEXT NOT NULL,
                subject TEXT NOT NULL,
                classes TEXT, -- JSON array of class_ids
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)
        print("  ✓ teachers 表创建成功")
        
    def create_classes_table(self):
        """创建班级表"""
        print("创建 classes 表...")
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS classes (
                class_id TEXT PRIMARY KEY,
                class_name TEXT NOT NULL,
                grade INTEGER NOT NULL,
                teacher_id TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (teacher_id) REFERENCES teachers(teacher_id)
            )
        """)
        print("  ✓ classes 表创建成功")
        
    def create_skill_points_table(self):
        """创建技能点表"""
        print("创建 skill_points 表...")
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS skill_points (
                point_id TEXT PRIMARY KEY,
                parent_id TEXT,
                name TEXT NOT NULL,
                category TEXT CHECK(category IN (
                    '知识概念',
                    '解题方法',
                    '题型模式',
                    '计算技巧',
                    '思维策略',
                    '应试技巧'
                )),
                difficulty_level INTEGER CHECK(difficulty_level BETWEEN 1 AND 5),
                description TEXT,
                typical_errors TEXT,
                key_insights TEXT,
                practice_tips TEXT,
                prerequisites TEXT,
                related_points TEXT,
                FOREIGN KEY (parent_id) REFERENCES skill_points(point_id)
            )
        """)
        
        # 创建索引
        self.cursor.execute("CREATE INDEX IF NOT EXISTS idx_skill_category ON skill_points(category)")
        self.cursor.execute("CREATE INDEX IF NOT EXISTS idx_skill_difficulty ON skill_points(difficulty_level)")
        print("  ✓ skill_points 表和索引创建成功")
        
    def create_question_bank_table(self):
        """创建题库表"""
        print("创建 question_bank 表...")
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS question_bank (
                question_id TEXT PRIMARY KEY,
                subject TEXT DEFAULT '数学',
                content TEXT NOT NULL,
                question_type TEXT CHECK(question_type IN ('单选题', '多选题', '填空题', '解答题')),
                difficulty INTEGER CHECK(difficulty BETWEEN 1 AND 5),
                cognitive_difficulty INTEGER CHECK(cognitive_difficulty BETWEEN 1 AND 5),
                points INTEGER NOT NULL,
                
                -- 答案相关
                solution TEXT NOT NULL,
                solution_detail TEXT,
                answer_options TEXT,
                correct_options TEXT,
                
                -- 题目特征
                common_mistakes TEXT,
                solving_time INTEGER,
                sub_questions INTEGER DEFAULT 1,
                
                -- 题目场景特征
                original_position INTEGER,
                position_category TEXT CHECK(position_category IN ('开篇题', '过渡题', '压轴题')),
                original_exam_id TEXT,
                original_year INTEGER,
                original_province TEXT,
                exam_type TEXT CHECK(exam_type IN ('高考', '模拟考', '联考', '周测')),
                time_pressure_level INTEGER CHECK(time_pressure_level BETWEEN 1 AND 5),
                
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)
        print("  ✓ question_bank 表创建成功")
        
    def create_question_skills_table(self):
        """创建题目-技能关联表"""
        print("创建 question_skills 表...")
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS question_skills (
                question_id TEXT,
                point_id TEXT,
                weight REAL CHECK(weight BETWEEN 0 AND 1),
                is_primary BOOLEAN DEFAULT FALSE,
                skill_usage TEXT,
                PRIMARY KEY (question_id, point_id),
                FOREIGN KEY (question_id) REFERENCES question_bank(question_id),
                FOREIGN KEY (point_id) REFERENCES skill_points(point_id)
            )
        """)
        print("  ✓ question_skills 表创建成功")
        
    def create_exams_table(self):
        """创建考试表"""
        print("创建 exams 表...")
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS exams (
                exam_id TEXT PRIMARY KEY,
                exam_name TEXT NOT NULL,
                exam_date DATE NOT NULL,
                exam_type TEXT CHECK(exam_type IN ('周测', '月考', '期中', '期末', '模拟考')),
                subject TEXT DEFAULT '数学',
                full_score INTEGER DEFAULT 150,
                class_id TEXT,
                exam_paper_url TEXT,
                duration INTEGER,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (class_id) REFERENCES classes(class_id)
            )
        """)
        print("  ✓ exams 表创建成功")
        
    def create_exam_questions_table(self):
        """创建考试题目表"""
        print("创建 exam_questions 表...")
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS exam_questions (
                exam_id TEXT,
                question_id TEXT,
                question_number INTEGER NOT NULL,
                question_position TEXT DEFAULT NULL,
                section TEXT NOT NULL,
                points INTEGER NOT NULL,
                PRIMARY KEY (exam_id, question_id),
                FOREIGN KEY (exam_id) REFERENCES exams(exam_id),
                FOREIGN KEY (question_id) REFERENCES question_bank(question_id)
            )
        """)
        print("  ✓ exam_questions 表创建成功")
        
    def create_exam_scores_table(self):
        """创建考试成绩表"""
        print("创建 exam_scores 表...")
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS exam_scores (
                score_id TEXT PRIMARY KEY,
                student_id TEXT NOT NULL,
                exam_id TEXT NOT NULL,
                total_score REAL NOT NULL,
                class_rank INTEGER,
                rank_change INTEGER,
                answer_sheet_url TEXT,
                time_used INTEGER,
                submitted_at TIMESTAMP,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (student_id) REFERENCES students(student_id),
                FOREIGN KEY (exam_id) REFERENCES exams(exam_id)
            )
        """)
        print("  ✓ exam_scores 表创建成功")
        
    def create_practice_records_table(self):
        """创建练习记录表"""
        print("创建 practice_records 表...")
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS practice_records (
                record_id TEXT PRIMARY KEY,
                student_id TEXT NOT NULL,
                question_id TEXT NOT NULL,
                practice_type TEXT CHECK(practice_type IN ('考试', '作业', '自主练习', '推荐练习')),
                exam_id TEXT,
                question_position INTEGER,
                
                -- 答题情况
                start_time TIMESTAMP,
                end_time TIMESTAMP,
                time_spent INTEGER,
                answer TEXT,
                is_correct BOOLEAN,
                score_earned REAL,
                
                -- 情境因素
                error_category TEXT CHECK(error_category IN ('知识性', '粗心', '审题', '计算', '涂卡', '超时', NULL)),
                time_of_attempt INTEGER,
                revision_count INTEGER DEFAULT 0,
                confidence_level INTEGER CHECK(confidence_level BETWEEN 1 AND 5),
                context_notes TEXT,
                
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (student_id) REFERENCES students(student_id),
                FOREIGN KEY (question_id) REFERENCES question_bank(question_id),
                FOREIGN KEY (exam_id) REFERENCES exams(exam_id)
            )
        """)
        print("  ✓ practice_records 表创建成功")
        
    def check_students_table(self):
        """检查学生表是否存在"""
        self.cursor.execute("""
            SELECT name FROM sqlite_master 
            WHERE type='table' AND name='students'
        """)
        if self.cursor.fetchone():
            # 获取学生数量
            self.cursor.execute("SELECT COUNT(*) FROM students")
            count = self.cursor.fetchone()[0]
            print(f"✓ 检测到 students 表已存在，包含 {count} 条学生记录")
            return True
        else:
            print("✗ 未检测到 students 表")
            return False
        
    
        
    def insert_sample_data(self):
        """插入一些示例数据"""
        print("\n插入示例数据...")
        
        # 插入示例教师
        self.cursor.execute("""
            INSERT OR IGNORE INTO teachers (teacher_id, name, subject, classes)
            VALUES ('T001', '王老师', '数学', '["2024-C1"]')
        """)
        
        # 插入示例班级
        self.cursor.execute("""
            INSERT OR IGNORE INTO classes (class_id, class_name, grade, teacher_id)
            VALUES ('2024-C1', '高三(1)班', 12, 'T001')
        """)
        
        print("  ✓ 示例数据插入成功")
        
    def create_all_tables(self):
        """创建所有表"""
        print("开始创建数据库架构...")
        print("=" * 50)
        
        # 检查学生表
        if not self.check_students_table():
            print("警告：students 表不存在，某些外键约束可能会失败")
            print("请确保已经创建并填充了 students 表")
            return False
        
        try:
            # 按照依赖顺序创建表
            self.create_teachers_table()
            self.create_classes_table()
            self.create_skill_points_table()
            self.create_question_bank_table()
            self.create_question_skills_table()
            self.create_exams_table()
            self.create_exam_questions_table()
            self.create_exam_scores_table()
            self.create_practice_records_table()

            
            # 插入示例数据
            self.insert_sample_data()
            
            # 提交事务
            self.conn.commit()
            
            print("=" * 50)
            print("✓ 所有表创建成功！")
            
            # 显示数据库概况
            self.show_database_info()
            
            return True
            
        except Exception as e:
            print(f"\n✗ 创建表时出错：{e}")
            self.conn.rollback()
            return False
            
    def show_database_info(self):
        """显示数据库信息"""
        print("\n数据库表概况：")
        self.cursor.execute("""
            SELECT name FROM sqlite_master 
            WHERE type='table' 
            ORDER BY name
        """)
        tables = self.cursor.fetchall()
        for table in tables:
            self.cursor.execute(f"SELECT COUNT(*) FROM {table[0]}")
            count = self.cursor.fetchone()[0]
            print(f"  - {table[0]}: {count} 条记录")
            
  
    def close(self):
        """关闭数据库连接"""
        self.conn.close()


# 主程序
if __name__ == "__main__":
    # 创建数据库架构
    db_creator = DatabaseSchemaCreator("school_management.db")
    
    try:
        # 创建所有表
        success = db_creator.create_all_tables()
        
        if success:
            print("\n数据库架构创建完成！")
            
        
    finally:
        db_creator.close()

开始创建数据库架构...
✓ 检测到 students 表已存在，包含 30 条学生记录
创建 teachers 表...
  ✓ teachers 表创建成功
创建 classes 表...
  ✓ classes 表创建成功
创建 skill_points 表...
  ✓ skill_points 表和索引创建成功
创建 question_bank 表...
  ✓ question_bank 表创建成功
创建 question_skills 表...
  ✓ question_skills 表创建成功
创建 exams 表...
  ✓ exams 表创建成功
创建 exam_questions 表...
  ✓ exam_questions 表创建成功
创建 exam_scores 表...
  ✓ exam_scores 表创建成功
创建 practice_records 表...
  ✓ practice_records 表创建成功

插入示例数据...
  ✓ 示例数据插入成功
✓ 所有表创建成功！

数据库表概况：
  - classes: 1 条记录
  - exam_questions: 0 条记录
  - exam_scores: 0 条记录
  - exams: 0 条记录
  - practice_records: 0 条记录
  - question_bank: 0 条记录
  - question_skills: 0 条记录
  - skill_points: 0 条记录
  - students: 30 条记录
  - teachers: 1 条记录

数据库架构创建完成！



# 考试模拟与批改系统

## 这个notebook将完成以下任务：
## 1. 创建测试考试
## 2. 生成30个学生的答卷
## 3. 批改答卷（第一份真实批改，其余模拟）
## 4. 将结果存入数据库
## 5. 生成本次考试的教师报告，包含每个学生的表现分析和练习题推荐




## 1. 导入必要的库和初始化



In [5]:
import os
import random
import sqlite3
import json
from datetime import datetime, date
from PIL import Image
from pathlib import Path
import base64
from dotenv import load_dotenv
from openai import OpenAI

# 加载环境变量
load_dotenv()

# 初始化OpenAI客户端
client = OpenAI()

# 数据库路径
DB_PATH = "school_management.db"



### 2. 定义题目信息和评分标准


In [6]:
# 定义本次考试的题目信息
EXAM_QUESTIONS = [
    {
        "question_id": "Q2024_TEST_001",
        "question_number": 1,
        "section": "单选题",
        "question_position": "开篇第一题",
        "points": 10,
        "correct_answer": "C",
        "skill_points": ["SP001", "SP101"]  # 基础概念 + 计算技巧
    },
    {
        "question_id": "Q2024_TEST_002",
        "question_number": 2,
        "section": "单选题",
        "question_position": None,
        "points": 10,
        "correct_answer": "C",
        "skill_points": ["SP002", "SP201"]  # 中等概念 + 解题方法
    },
    {
        "question_id": "Q2024_TEST_003",
        "question_number": 3,
        "section": "多选题",
        "question_position": "多选题最后一道",
        "points": 20,
        "correct_answer": "ABD",
        "skill_points": ["SP003", "SP301"]  # 综合应用
    },
    {
        "question_id": "Q2024_TEST_004",
        "question_number": 4,
        "section": "填空题",
        "question_position": None,
        "points": 10,
        "correct_answer": "2.5",
        "skill_points": ["SP004"]
    },
    {
        "question_id": "Q2024_TEST_005",
        "question_number": 5,
        "section": "解答题",
        "question_position": "本卷压轴题",
        "points": 50,
        "correct_answer": "完整解答",
        "skill_points": ["SP005", "SP401", "SP501"]  # 高级技能组合
    }
]

# 总分
TOTAL_POINTS = sum(q["points"] for q in EXAM_QUESTIONS)



### 3. 【函数】生成学生的答卷


In [7]:
def generate_random_answer_sheet(base_path, output_path, student_id, force_pattern="normal"):
    """
    生成随机答卷并返回答题情况
    
    Returns:
        dict: 包含每道题的答题情况和得分
    """
    # 定义每道题的不同作答情况及其权重
    questions = {
        "q1_single_choice": {
            "answered_correctly.jpg": {"weight": 0.9, "is_correct": True, "score_rate": 1.0},
            "answered_wrongly.jpg": {"weight": 0.1, "is_correct": False, "score_rate": 0.0}
        },
        "q2_single_choice": {
            "answered_correctly.jpg": {"weight": 0.7, "is_correct": True, "score_rate": 1.0},
            "answered_wrongly.jpg": {"weight": 0.3, "is_correct": False, "score_rate": 0.0}
        },
        "q3_multiple_choice": {
            "answered_correctly.jpg": {"weight": 0.4, "is_correct": True, "score_rate": 1.0},
            "answered_partially_correct.jpg": {"weight": 0.6, "is_correct": False, "score_rate": 0.5},
            "answered_wrongly.jpg": {"weight": 0.1, "is_correct": False, "score_rate": 0.0}
        },
        "q4_completion": {
            "answered_correctly.jpg": {"weight": 0.7, "is_correct": True, "score_rate": 1.0},
            "answered_wrongly.jpg": {"weight": 0.3, "is_correct": False, "score_rate": 0.0}
        },
        "q5_solution": {
            "answered_correctly.jpg": {"weight": 0.5, "is_correct": True, "score_rate": 1.0},
            "answered_partially_correct.jpg": {"weight": 0.5, "is_correct": False, "score_rate": 0.3}
        }
    }
    
    # 记录答题情况
    answer_records = []
    selected_images = []
    
    for i, (q_name, answers) in enumerate(questions.items()):
        # 如果指定了特殊模式
        if force_pattern == "first_wrong_others_correct":
            if i == 0:  # 第一题强制错误
                selected_answer = "answered_wrongly.jpg"
            else:  # 其他题强制正确
                selected_answer = "answered_correctly.jpg"
        else:
            # 正常随机选择
            answer_files = list(answers.keys())
            weights = [answers[f]["weight"] for f in answer_files]
            selected_answer = random.choices(answer_files, weights=weights, k=1)[0]
        
        img_path = f"{base_path}/{q_name}_{selected_answer}"
        selected_images.append(img_path)
        
        # 记录答题情况
        answer_info = answers[selected_answer]
        question_info = EXAM_QUESTIONS[i]
        
        # 特殊处理错误类型
        error_category = None
        if not answer_info["is_correct"]:
            if force_pattern == "first_wrong_others_correct" and i == 0:
                error_category = "粗心"  # 第一题错误标记为粗心
            else:
                error_category = "知识性" if "wrongly" in selected_answer else "粗心"
        
        answer_records.append({
            "question_id": question_info["question_id"],
            "question_number": question_info["question_number"],
            "is_correct": answer_info["is_correct"],
            "score_earned": question_info["points"] * answer_info["score_rate"],
            "max_points": question_info["points"],
            "answer_type": "not_llm_graded",
            "error_category": error_category,
            "comment": None
        })
    
    # 生成答卷图片
    images = [Image.open(img_path) for img_path in selected_images]
    width = min(img.size[0] for img in images)
    heights = []
    resized_images = []
    
    for img in images:
        height = int(img.size[1] * width / img.size[0])
        heights.append(height)
        resized_images.append(img.resize((width, height)))
    
    total_height = sum(heights)
    merged_img = Image.new('RGB', (width, total_height))
    
    y_offset = 0
    for img in resized_images:
        merged_img.paste(img, (0, y_offset))
        y_offset += img.size[1]
    
    merged_img.save(output_path)
    
    return answer_records, selected_images


### 4. 【函数】批改学生试卷


In [8]:
from pydantic import BaseModel
from typing import List, Optional, Literal
from openai import OpenAI

# 定义结构化输出的数据模型
class QuestionGrading(BaseModel):
    is_correct: bool
    score_rate: float  # 0.0 到 1.0
    error_category: Optional[Literal["知识性", "粗心"]] = None  # 只能是"知识性"或"粗心"，或None
    comment: str


# 客户端
client = OpenAI()

def grade_single_question(image_path: str, question_info: dict) -> dict:
    """
    批改单个题目
    
    Args:
        image_path: 题目图片路径
        question_info: 题目信息字典
    
    Returns:
        标准格式的答题记录
    """
    try:
        # 创建文件并获取file_id
        def create_file(file_path):
            with open(file_path, "rb") as file_content:
                result = client.files.create(
                    file=file_content,
                    purpose="vision",
                )
                return result.id
        
        file_id = create_file(image_path)
        # 根据题型构建提示
        if question_info["section"] == "单选题":
            prompt = f"这是一道单选题（{question_info['points']}分）。请判断学生是否答对，给出得分率（0或1）。"
        elif question_info["section"] == "多选题":
            prompt = f"这是一道多选题（{question_info['points']}分）。完全正确得满分，部分正确得0.6，全错得0。"
        elif question_info["section"] == "填空题":
            prompt = f"这是一道填空题（{question_info['points']}分）。请判断答案是否正确。"
        elif question_info["section"] == "解答题":
            prompt = f"这是一道解答题（{question_info['points']}分）。请评估解答过程，可以给部分分（如0.6表示得60%的分数）。"
        
        # 使用结构化输出
        response = client.responses.parse(
            model="o4-mini",
            input=[
                {
                    "role": "user",
                    "content": [
                        {"type": "input_text", "text": prompt},
                        {
                            "type": "input_image",
                            "file_id": file_id,
                            "detail": "high",
                        }
                    ]
                }
            ],
            text_format=QuestionGrading,
        )
        
        # 获取结构化输出
        grading = response.output_parsed
        print('grading', grading)
        
        # 转换为标准格式
        return {
            "question_id": question_info["question_id"],
            "question_number": question_info["question_number"],
            "is_correct": grading.is_correct,
            "score_earned": question_info["points"] * grading.score_rate,
            "max_points": question_info["points"],
            "answer_type": "llm_graded",
            "error_category": grading.error_category,
            "comment": grading.comment
        }
        
    except Exception as e:
        print(f"批改第{question_info['question_number']}题时出错: {e}")
        # 返回默认的失败结果
        return {
            "question_id": question_info["question_id"],
            "question_number": question_info["question_number"],
            "is_correct": False,
            "score_earned": 0,
            "max_points": question_info["points"],
            "answer_type": "grading_failed",
            "error_category": "系统错误",
            "comment": f"批改失败: {str(e)}"
        }



def grade_answer_sheet_with_llm(selected_images: List[str], exam_questions: List[dict]) -> List[dict]:
    """
    使用LLM串行批改答卷
    
    Args:
        selected_images: 题目图片路径列表
        exam_questions: 考试题目信息列表
    
    Returns:
        list: 与generate_random_answer_sheet返回格式相同的答题记录列表
    """
    try:
        results = []
        
        # 串行批改每道题
        for i, (image_path, question_info) in enumerate(zip(selected_images, exam_questions)):
            print(f"    正在批改第{i+1}题...")
            result = grade_single_question(image_path, question_info)
            results.append(result)
            
            # 打印批改结果
            status = "✓" if result["is_correct"] else "✗"
            print(f"      {status} 得分：{result['score_earned']}/{result['max_points']}")
            if result.get("comment"):
                print(f"      评语：{result['comment']}")
        
        return results
        
    except Exception as e:
        print(f"LLM批改失败: {e}")
        return None



### 5.【函数】数据库操作函数



In [9]:
def create_exam_in_db(conn, exam_id, exam_name, exam_date, class_id):
    """创建考试记录"""
    cursor = conn.cursor()
    cursor.execute('''
        INSERT INTO exams (exam_id, exam_name, exam_date, exam_type, 
                          subject, full_score, class_id, duration)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', (exam_id, exam_name, exam_date, '周测', '数学', TOTAL_POINTS, class_id, 120))
    
    # 插入考试题目
    for q in EXAM_QUESTIONS:
        cursor.execute('''
            INSERT INTO exam_questions (exam_id, question_id, question_number, 
                                      question_position, section, points)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (exam_id, q["question_id"], q["question_number"], 
              q["question_position"], q["section"], q["points"]))
    
    conn.commit()

def save_exam_scores_base(conn, exam_id, student_id, answer_records):
    """保存考试基本成绩信息，不包含排名
    
    Args:
        conn: 数据库连接
        exam_id: 考试ID
        student_id: 学生ID
        answer_records: 答题记录列表
    """
    cursor = conn.cursor()
    
    # 计算总分
    total_score = sum(record["score_earned"] for record in answer_records)
    
    # 插入考试成绩
    score_id = f"SCORE_{exam_id}_{student_id}"
    cursor.execute('''
        INSERT INTO exam_scores (score_id, student_id, exam_id, total_score, 
                               answer_sheet_url)
        VALUES (?, ?, ?, ?, ?)
    ''', (score_id, student_id, exam_id, total_score,
          f"answer_sheets/{exam_id}/{student_id}.jpg"))
    
    # 插入每道题的答题记录
    for i, record in enumerate(answer_records):
        record_id = f"REC_{exam_id}_{student_id}_Q{i+1}"
        cursor.execute('''
            INSERT INTO practice_records (record_id, student_id, question_id, 
                                        practice_type, exam_id, question_position,
                                        is_correct, score_earned, error_category)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (record_id, student_id, record["question_id"], '考试', exam_id,
              record["question_number"], record["is_correct"], 
              record["score_earned"], record["error_category"]))
    
    conn.commit()

def update_exam_rank(conn, exam_id, student_id, rank):
    """更新考试排名信息
    
    Args:
        conn: 数据库连接
        exam_id: 考试ID
        student_id: 学生ID
        rank: 排名
    """
    cursor = conn.cursor()
    
    cursor.execute('''
        UPDATE exam_scores 
        SET class_rank = ?
        WHERE exam_id = ? AND student_id = ?
    ''', (rank, exam_id, student_id))
    
    conn.commit()


### 6. 主程序：生成30个学生的考试数据


In [10]:
# 设置路径
base_path = "answered_questions_pool"
output_dir = "exam_results"
os.makedirs(output_dir, exist_ok=True)

# 考试信息
exam_id = "EXAM_2024_TEST_001"
exam_name = "2024年秋季学期第10周周测"
exam_date = datetime.now().strftime("%Y-%m-%d")
class_id = "2024-C1"

# 连接数据库
conn = sqlite3.connect(DB_PATH)

# 创建考试记录
print("创建考试记录...")
create_exam_in_db(conn, exam_id, exam_name, exam_date, class_id)

# 获取班级学生列表
cursor = conn.cursor()
cursor.execute('SELECT student_id, name FROM students WHERE class_id = ? ORDER BY student_id', 
               (class_id,))
students = cursor.fetchall()[:30]  # 只取前30个学生

# 存储所有学生的成绩用于排名
all_scores = []

print(f"\n开始生成{len(students)}个学生的答卷...")

# %% 
# 为每个学生生成答卷和成绩
for idx, (student_id, student_name) in enumerate(students):
    print(f"\n处理学生 {idx+1}/{len(students)}: {student_name} ({student_id})")
    
    # 生成答卷
    answer_sheet_path = f"{output_dir}/{student_id}.jpg"
    
    # 第3个学生（索引2）使用特殊模式
    if idx == 2:
        print("  *** 生成特殊案例：只错第一题的学生 ***")
        answer_records, selected_images = generate_random_answer_sheet(
            base_path, answer_sheet_path, student_id, 
            force_pattern="first_wrong_others_correct"
        )
    elif idx != 0:
        answer_records, selected_images = generate_random_answer_sheet(
            base_path, answer_sheet_path, student_id
        )
    
    # 第一个学生使用LLM批改
    if idx == 0:
        answer_records, selected_images = generate_random_answer_sheet(
            base_path, answer_sheet_path, student_id
        )
        print("  使用LLM批改第一份答卷...")
        try:
            llm_feedback = grade_answer_sheet_with_llm(selected_images, EXAM_QUESTIONS)
            answer_records = llm_feedback
            #print(f"  LLM批改结果：\n{llm_feedback[:200]}...")
        except Exception as e:
            print(f"  LLM批改出错：{e}")

    
    # 计算总分
    total_score = sum(record["score_earned"] for record in answer_records)
    all_scores.append((student_id, total_score))
    
    # 打印答题情况
    print(f"  总分：{total_score}/{TOTAL_POINTS}")
    for record in answer_records:
        status = "✓" if record["is_correct"] else "✗"
        print(f"    第{record['question_number']}题 {status} 得分：{record['score_earned']}/{record['max_points']}")
    
    # 特殊案例的额外说明
    if idx == 2:
        print("  【特殊模式分析】：该学生展现了'会难题却错简单题'的典型模式")
        print("  可能原因：考试开始时紧张、粗心大意、或审题不仔细")
    # 保存基本成绩信息
    save_exam_scores_base(conn, exam_id, student_id, answer_records)
# %%
# 计算排名
print("\n计算班级排名...")
all_scores.sort(key=lambda x: x[1], reverse=True)
score_ranks = {student_id: rank+1 for rank, (student_id, score) in enumerate(all_scores)}

# 保存所有成绩到数据库
print("\n保存成绩到数据库...")
for student_id, student_name in students:
    # 更新排名
    rank = score_ranks[student_id]
    update_exam_rank(conn, exam_id, student_id, rank)

# %%
# 显示成绩统计
print("\n成绩统计：")
print(f"参考人数：{len(students)}")
print(f"最高分：{all_scores[0][1]}")
print(f"最低分：{all_scores[-1][1]}")
print(f"平均分：{sum(score for _, score in all_scores) / len(all_scores):.1f}")

print("\n前5名：")
for rank, (student_id, score) in enumerate(all_scores[:5], 1):
    cursor.execute('SELECT name FROM students WHERE student_id = ?', (student_id,))
    name = cursor.fetchone()[0]
    print(f"  第{rank}名：{name} - {score}分")

# %%
# 关闭数据库连接
conn.close()
print("\n考试数据生成完成！")


创建考试记录...

开始生成30个学生的答卷...

处理学生 1/30: 黄婷 (2024001)
  使用LLM批改第一份答卷...
    正在批改第1题...
grading is_correct=True score_rate=1.0 error_category=None comment=''
      ✓ 得分：10.0/10
    正在批改第2题...
grading is_correct=True score_rate=1.0 error_category=None comment='正确，AF=5，对应选项C。'
      ✓ 得分：10.0/10
      评语：正确，AF=5，对应选项C。
    正在批改第3题...
grading is_correct=True score_rate=1.0 error_category=None comment='正确选项为 A、B、D。'
      ✓ 得分：20.0/20
      评语：正确选项为 A、B、D。
    正在批改第4题...
grading is_correct=True score_rate=1.0 error_category=None comment='代入两球中心坐标可得方程 4(4−r)^2+(9−2r)^2=4r^2，解得 r=2.5\u2009cm，符合最优布置，答案正确。'
      ✓ 得分：10.0/10
      评语：代入两球中心坐标可得方程 4(4−r)^2+(9−2r)^2=4r^2，解得 r=2.5 cm，符合最优布置，答案正确。
    正在批改第5题...
grading is_correct=True score_rate=0.96 error_category=None comment='第一、二问计算正确；第三问思路清晰，通过递推化简到组合数不等式并加以证明，虽书写略显简略但要点完整，可酌情给满分或略减少1～2分。'
      ✓ 得分：48.0/50
      评语：第一、二问计算正确；第三问思路清晰，通过递推化简到组合数不等式并加以证明，虽书写略显简略但要点完整，可酌情给满分或略减少1～2分。
  总分：98.0/100
    第1题 ✓ 得分：10.0/10
    第2题 ✓ 得分：10.0/10
    第3


### 7. 验证数据


### 重新连接数据库验证数据

In [11]:



conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# 查询考试信息
print("考试信息：")
cursor.execute('SELECT * FROM exams WHERE exam_id = ?', (exam_id,))
exam_info = cursor.fetchone()
print(f"  考试ID：{exam_info[0]}")
print(f"  考试名称：{exam_info[1]}")
print(f"  考试日期：{exam_info[2]}")

# 查询成绩分布
print("\n成绩分布：")
cursor.execute('''
    SELECT 
        CASE 
            WHEN total_score >= 90 THEN '优秀(90%+)'
            WHEN total_score >= 70 THEN '良好(70-90%)'
            WHEN total_score >= 50 THEN '及格(50-70%)'
            ELSE '不及格(<50%)'
        END as grade,
        COUNT(*) as count
    FROM exam_scores
    WHERE exam_id = ?
    GROUP BY grade
    ORDER BY MIN(total_score) DESC
''', (exam_id,))

for grade, count in cursor.fetchall():
    print(f"  {grade}: {count}人")

conn.close()

# %% [markdown]
# ## 完成！
# 
# 现在我们已经：
# 1. 创建了一次测试考试
# 2. 生成了30个学生的随机答卷
# 3. 模拟了批改过程（第一份使用LLM）
# 4. 将所有数据存入了数据库
# 
# 下一步可以：
# - 分析学生的答题模式
# - 识别知识薄弱点
# - 生成个性化推荐

考试信息：
  考试ID：EXAM_2024_TEST_001
  考试名称：2024年秋季学期第10周周测
  考试日期：2025-06-22

成绩分布：
  优秀(90%+): 11人
  良好(70-90%): 4人
  及格(50-70%): 10人
  不及格(<50%): 5人


# 实例化技能点表、题库表以及题目技能点对照表

In [12]:
import sqlite3
import random
import json
from datetime import datetime
from typing import List, Dict, Tuple

class MockDataGenerator:
    def __init__(self, db_path: str = "school_management.db"):
        self.db_path = db_path
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()
        
        # 预定义的技能点数据
        self.skill_data = {
            "知识概念": [
                ("二次函数", "二次函数的图像与性质", 2),
                ("导数概念", "导数的定义与计算", 3),
                ("三角函数", "三角函数的图像与性质", 2),
                ("数列通项", "等差等比数列的通项公式", 2),
                ("立体几何", "空间向量与立体几何", 3),
                ("概率统计", "概率分布与期望方差", 3),
                ("不等式", "基本不等式及其应用", 2),
                ("函数单调性", "函数单调性的判断与应用", 2),
                ("极值最值", "函数的极值与最值问题", 3),
                ("向量运算", "平面向量的运算与应用", 2),
                ("圆锥曲线", "椭圆双曲线抛物线的性质", 3),
                ("复数", "复数的运算与几何意义", 2),
                ("排列组合", "排列组合的计算方法", 2),
                ("二项式定理", "二项式展开与系数", 2),
                ("对数函数", "对数函数的性质与运算", 2),
                ("指数函数", "指数函数的性质与运算", 2),
                ("三角恒等变换", "三角恒等式的证明与应用", 3),
                ("解三角形", "正弦定理余弦定理的应用", 2),
                ("直线方程", "直线方程的各种形式", 1),
                ("定积分", "定积分的计算与应用", 3),
            ],
            "解题方法": [
                ("换元法", "通过换元简化问题", 3),
                ("数形结合", "利用图形直观解决代数问题", 3),
                ("分类讨论", "按条件分类解决问题", 3),
                ("构造函数法", "构造辅助函数解决问题", 4),
                ("待定系数法", "设定未知系数求解", 2),
                ("反证法", "假设相反结论推出矛盾", 3),
                ("数学归纳法", "用归纳法证明数列命题", 3),
                ("分离变量法", "将变量分离到不等式两端", 3),
                ("配方法", "通过配方简化表达式", 2),
                ("因式分解", "多项式的因式分解技巧", 2),
                ("放缩法", "通过放缩证明不等式", 4),
                ("导数法", "利用导数研究函数性质", 3),
                ("向量法", "用向量方法解决几何问题", 3),
                ("坐标法", "建立坐标系解决几何问题", 3),
                ("参数方程法", "引入参数简化问题", 3),
            ],
            "题型模式": [
                ("恒成立问题", "不等式恒成立求参数范围", 4),
                ("存在性问题", "存在某个值使条件成立", 4),
                ("最值问题", "求函数或表达式的最值", 3),
                ("动点轨迹", "求动点的轨迹方程", 3),
                ("定点定值", "证明某个量为定值", 3),
                ("参数范围", "求参数的取值范围", 3),
                ("零点问题", "函数零点的个数与分布", 3),
                ("单调性问题", "判断或应用函数单调性", 2),
                ("对称性问题", "利用对称性简化计算", 2),
                ("周期性问题", "利用周期性解决问题", 2),
                ("数列求和", "各种数列求和方法", 3),
                ("递推数列", "递推关系求通项", 3),
            ],
            "计算技巧": [
                ("裂项相消", "分式数列求和的裂项技巧", 3),
                ("错位相减", "等比等差混合数列求和", 3),
                ("配方技巧", "二次式的配方技巧", 2),
                ("三角恒等变换", "三角函数的恒等变换", 3),
                ("分组求和", "将数列分组后求和", 2),
                ("倒序相加", "利用倒序简化求和", 2),
                ("有理化", "分母有理化技巧", 2),
                ("整体代换", "整体代换简化计算", 2),
                ("对称性简化", "利用对称性简化积分", 3),
                ("特殊值代入", "代入特殊值简化计算", 2),
            ],
            "思维策略": [
                ("逆向思维", "从结论出发反推条件", 3),
                ("极端值检验", "用极端情况验证结论", 2),
                ("特殊化思想", "从特殊到一般的推理", 3),
                ("类比推理", "通过类比发现规律", 3),
                ("归纳猜想", "从特例归纳一般规律", 3),
                ("化归思想", "将复杂问题化归为简单问题", 3),
                ("整体思想", "从整体角度考虑问题", 2),
                ("方程思想", "建立方程解决问题", 2),
            ],
            "应试技巧": [
                ("选择题排除法", "通过排除错误选项确定答案", 1),
                ("选择题特值法", "代入特殊值验证选项", 1),
                ("填空题边界检验", "检验边界情况", 2),
                ("解答题得分策略", "确保基础分不丢失", 2),
                ("时间分配技巧", "合理分配考试时间", 2),
                ("审题技巧", "准确理解题目要求", 1),
                ("检查策略", "高效检查答案", 1),
            ]
        }
        
    def create_skill_points(self):
        """创建技能点数据"""
        print("生成技能点数据...")
        
        skill_id_counter = 1
        skill_ids = {}  # 存储技能点名称到ID的映射
        
        for category, skills in self.skill_data.items():
            for skill_name, description, difficulty in skills:
                skill_id = f"SP{skill_id_counter:03d}"
                skill_ids[skill_name] = skill_id
                
                # 随机添加一些额外信息
                typical_errors = self._generate_typical_errors(category, skill_name)
                key_insights = self._generate_key_insights(category, skill_name)
                practice_tips = self._generate_practice_tips(category, skill_name)
                
                self.cursor.execute('''
                    INSERT INTO skill_points 
                    (point_id, name, category, difficulty_level, description,
                     typical_errors, key_insights, practice_tips)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                ''', (skill_id, skill_name, category, difficulty, description,
                      typical_errors, key_insights, practice_tips))
                
                skill_id_counter += 1
        
        self.conn.commit()
        print(f"  已创建 {skill_id_counter - 1} 个技能点")
        return skill_ids
    
    def create_questions(self, skill_ids: Dict[str, str], num_questions: int = 250):
        """创建题目数据"""
        print(f"\n生成 {num_questions} 道题目...")
        
        question_ids = []
        
        # 题型分布
        question_types = [
            ("单选题", 100, 5),
            ("多选题", 50, 10),
            ("填空题", 50, 5),
            ("解答题", 50, 25)
        ]
        
        question_counter = 1
        
        for q_type, count, points in question_types:
            for i in range(min(count, num_questions - question_counter + 1)):
                question_id = f"Q{datetime.now().year}_{question_counter:04d}"
                question_ids.append(question_id)
                
                # 生成题目属性
                difficulty = self._generate_difficulty_distribution()
                cognitive_difficulty = max(1, min(5, difficulty + random.randint(-1, 1)))
                
                # 生成位置相关属性
                original_position = random.randint(1, 22)
                position_category = self._get_position_category(original_position, q_type)
                
                # 生成其他属性
                solving_time = self._get_solving_time(q_type, difficulty)
                time_pressure_level = self._get_time_pressure_level(position_category, difficulty)
                
                # 题目内容（模拟）
                content = f"【{q_type}】这是第{question_counter}题的题目内容..."
                solution = f"这是第{question_counter}题的标准答案..."
                solution_detail = f"这是第{question_counter}题的详细解析..."
                
                # 对于选择题，生成选项
                answer_options = None
                correct_options = None
                if "选择" in q_type:
                    answer_options, correct_options = self._generate_options(q_type)
                
                self.cursor.execute('''
                    INSERT INTO question_bank
                    (question_id, subject, content, question_type, difficulty,
                     cognitive_difficulty, points, solution, solution_detail,
                     answer_options, correct_options, solving_time,
                     original_position, position_category, original_year,
                     original_province, exam_type, time_pressure_level)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', (question_id, '数学', content, q_type, difficulty,
                      cognitive_difficulty, points, solution, solution_detail,
                      answer_options, correct_options, solving_time,
                      original_position, position_category,
                      random.randint(2020, 2024), self._random_province(),
                      '高考', time_pressure_level))
                
                question_counter += 1
                if question_counter > num_questions:
                    break
            
            if question_counter > num_questions:
                break
        
        self.conn.commit()
        print(f"  已创建 {len(question_ids)} 道题目")
        return question_ids
    
    def create_question_skills(self, question_ids: List[str], skill_ids: Dict[str, str]):
        """创建题目-技能关联"""
        print("\n生成题目-技能关联数据...")
        
        all_skill_ids = list(skill_ids.values())
        relation_count = 0
        
        for question_id in question_ids:
            # 每道题关联3-5个技能
            num_skills = random.randint(3, 5)
            selected_skills = random.sample(all_skill_ids, num_skills)
            
            # 第一个技能作为主要技能
            for i, skill_id in enumerate(selected_skills):
                is_primary = (i == 0)
                weight = random.uniform(0.6, 1.0) if is_primary else random.uniform(0.2, 0.5)
                
                # 生成技能使用说明
                skill_usage = self._generate_skill_usage(skill_id, is_primary)
                
                self.cursor.execute('''
                    INSERT INTO question_skills
                    (question_id, point_id, weight, is_primary, skill_usage)
                    VALUES (?, ?, ?, ?, ?)
                ''', (question_id, skill_id, weight, is_primary, skill_usage))
                
                relation_count += 1
        
        self.conn.commit()
        print(f"  已创建 {relation_count} 条题目-技能关联")
    
    # 辅助方法
    def _generate_difficulty_distribution(self) -> int:
        """根据分布生成难度"""
        rand = random.random()
        if rand < 0.15:
            return 1
        elif rand < 0.30:
            return 2
        elif rand < 0.70:
            return 3
        elif rand < 0.90:
            return 4
        else:
            return 5
    
    def _get_position_category(self, position: int, q_type: str) -> str:
        """根据位置确定类别"""
        if position <= 3:
            return "开篇题"
        elif q_type == "解答题" and position >= 20:
            return "压轴题"
        else:
            return "过渡题"
    
    def _get_solving_time(self, q_type: str, difficulty: int) -> int:
        """获取建议解题时间"""
        base_time = {
            "单选题": 2,
            "多选题": 3,
            "填空题": 3,
            "解答题": 10
        }
        return base_time.get(q_type, 5) + difficulty
    
    def _get_time_pressure_level(self, position_category: str, difficulty: int) -> int:
        """获取时间压力等级"""
        if position_category == "压轴题":
            return min(5, difficulty + 1)
        elif position_category == "开篇题":
            return max(1, difficulty - 1)
        else:
            return difficulty
    
    def _random_province(self) -> str:
        """随机省份"""
        provinces = ["全国卷I", "全国卷II", "全国卷III", "北京卷", "上海卷", 
                    "浙江卷", "江苏卷", "山东卷", "广东卷", "天津卷"]
        return random.choice(provinces)
    
    def _generate_options(self, q_type: str) -> Tuple[str, str]:
        """生成选择题选项"""
        if q_type == "单选题":
            options = {"A": "选项A内容", "B": "选项B内容", 
                      "C": "选项C内容", "D": "选项D内容"}
            correct = random.choice(["A", "B", "C", "D"])
        else:  # 多选题
            options = {"A": "选项A内容", "B": "选项B内容", 
                      "C": "选项C内容", "D": "选项D内容"}
            num_correct = random.randint(2, 3)
            correct_list = random.sample(["A", "B", "C", "D"], num_correct)
            correct = "".join(sorted(correct_list))
        
        return json.dumps(options, ensure_ascii=False), correct
    
    def _generate_typical_errors(self, category: str, skill_name: str) -> str:
        """生成典型错误"""
        errors = {
            "知识概念": "概念理解不清，公式记忆错误",
            "解题方法": "方法选择不当，步骤遗漏",
            "题型模式": "模式识别错误，条件遗漏",
            "计算技巧": "计算过程出错，技巧运用不熟",
            "思维策略": "思维定势，考虑不全面",
            "应试技巧": "时间分配不当，审题不仔细"
        }
        return errors.get(category, "常见错误")
    
    def _generate_key_insights(self, category: str, skill_name: str) -> str:
        """生成关键洞察"""
        return f"{skill_name}的关键在于准确识别使用场景"
    
    def _generate_practice_tips(self, category: str, skill_name: str) -> str:
        """生成练习建议"""
        return f"多做{skill_name}相关的典型例题，总结规律"
    
    def _generate_skill_usage(self, skill_id: str, is_primary: bool) -> str:
        """生成技能使用说明"""
        if is_primary:
            return "本题主要考察该技能的综合运用"
        else:
            return "本题需要用到该技能作为辅助"
    
    def generate_all_data(self):
        """生成所有模拟数据"""
        print("开始生成模拟数据...")
        print("=" * 50)
        
        # 生成技能点
        skill_ids = self.create_skill_points()
        
        # 生成题目
        question_ids = self.create_questions(skill_ids, num_questions=250)
        
        # 生成题目-技能关联
        self.create_question_skills(question_ids, skill_ids)
        
        print("=" * 50)
        print("模拟数据生成完成！")
        
        # 显示统计信息
        self.show_statistics()
    
    def show_statistics(self):
        """显示生成的数据统计"""
        print("\n数据统计：")
        
        # 技能点统计
        self.cursor.execute("SELECT category, COUNT(*) FROM skill_points GROUP BY category")
        print("\n技能点分布：")
        for category, count in self.cursor.fetchall():
            print(f"  {category}: {count}个")
        
        # 题目统计
        self.cursor.execute("SELECT question_type, COUNT(*) FROM question_bank GROUP BY question_type")
        print("\n题目类型分布：")
        for q_type, count in self.cursor.fetchall():
            print(f"  {q_type}: {count}道")
        
        # 难度分布
        self.cursor.execute("SELECT difficulty, COUNT(*) FROM question_bank GROUP BY difficulty")
        print("\n题目难度分布：")
        for difficulty, count in self.cursor.fetchall():
            print(f"  难度{difficulty}: {count}道")
        
        # 关联统计
        self.cursor.execute("SELECT COUNT(*) FROM question_skills")
        total_relations = self.cursor.fetchone()[0]
        self.cursor.execute("SELECT COUNT(*) FROM question_skills WHERE is_primary = 1")
        primary_relations = self.cursor.fetchone()[0]
        print(f"\n题目-技能关联：")
        print(f"  总关联数: {total_relations}条")
        print(f"  主要技能: {primary_relations}条")
        print(f"  平均每题关联: {total_relations / 250:.1f}个技能")
    
    def close(self):
        """关闭数据库连接"""
        self.conn.close()


# 使用示例
if __name__ == "__main__":
    # 创建数据生成器
    generator = MockDataGenerator("school_management.db")
    
    try:
        # 生成所有数据
        generator.generate_all_data()
    finally:
        # 关闭数据库连接
        generator.close()

开始生成模拟数据...
生成技能点数据...
  已创建 72 个技能点

生成 250 道题目...
  已创建 250 道题目

生成题目-技能关联数据...
  已创建 1001 条题目-技能关联
模拟数据生成完成！

数据统计：

技能点分布：
  应试技巧: 7个
  思维策略: 8个
  知识概念: 20个
  解题方法: 15个
  计算技巧: 10个
  题型模式: 12个

题目类型分布：
  单选题: 100道
  填空题: 50道
  多选题: 50道
  解答题: 50道

题目难度分布：
  难度1: 36道
  难度2: 34道
  难度3: 104道
  难度4: 51道
  难度5: 25道

题目-技能关联：
  总关联数: 1001条
  主要技能: 250条
  平均每题关联: 4.0个技能


# 生成本次的提交给老师的考试报告，包含了每个同学的详细表现

In [13]:
import sqlite3
import pandas as pd
from datetime import datetime
import random
from typing import Dict, List, Tuple

class SimpleAnalysisSystem:
    def __init__(self, db_path: str = "school_management.db", exam_id: str = "EXAM_2024_TEST_001"):
        self.db_path = db_path
        self.exam_id = exam_id
        self.conn = sqlite3.connect(db_path)
        
    def analyze_exam_overview(self) -> Dict:
        """分析考试整体情况"""
        # 获取考试基本信息
        exam_info = self.conn.execute("""
            SELECT exam_name, exam_date, full_score 
            FROM exams WHERE exam_id = ?
        """, [self.exam_id]).fetchone()
        
        # 获取成绩统计
        stats = self.conn.execute("""
            SELECT 
                COUNT(*) as total_students,
                AVG(total_score) as avg_score,
                MAX(total_score) as max_score,
                MIN(total_score) as min_score,
                COUNT(CASE WHEN total_score >= ? * 0.9 THEN 1 END) as excellent_count,
                COUNT(CASE WHEN total_score >= ? * 0.6 AND total_score < ? * 0.9 THEN 1 END) as pass_count,
                COUNT(CASE WHEN total_score < ? * 0.6 THEN 1 END) as fail_count
            FROM exam_scores 
            WHERE exam_id = ?
        """, [exam_info[2], exam_info[2], exam_info[2], exam_info[2], self.exam_id]).fetchone()
        
        # 获取每道题的统计
        question_stats = pd.read_sql_query("""
            SELECT 
                pr.question_position as 题号,
                eq.section as 题型,
                eq.points as 分值,
                AVG(CASE WHEN pr.is_correct THEN 1.0 ELSE 0.0 END) as 正确率,
                COUNT(CASE WHEN pr.error_category = '知识性' THEN 1 END) as 知识性错误数,
                COUNT(CASE WHEN pr.error_category = '粗心' THEN 1 END) as 粗心错误数
            FROM practice_records pr
            JOIN exam_questions eq ON pr.exam_id = eq.exam_id AND pr.question_position = eq.question_number
            WHERE pr.exam_id = ?
            GROUP BY pr.question_position
            ORDER BY pr.question_position
        """, self.conn, params=[self.exam_id])
        
        return {
            'exam_name': exam_info[0],
            'exam_date': exam_info[1],
            'full_score': exam_info[2],
            'total_students': stats[0],
            'avg_score': round(stats[1], 1),
            'max_score': stats[2],
            'min_score': stats[3],
            'excellent_rate': round(stats[4] / stats[0] * 100, 1),
            'pass_rate': round((stats[4] + stats[5]) / stats[0] * 100, 1),
            'fail_count': stats[6],
            'question_stats': question_stats
        }
    
    def analyze_student_errors(self, student_id: str) -> Dict:
        """分析单个学生的错题情况"""
        # 获取学生基本信息
        student_info = self.conn.execute("""
            SELECT s.name, s.mbti, es.total_score, es.class_rank
            FROM students s
            JOIN exam_scores es ON s.student_id = es.student_id
            WHERE s.student_id = ? AND es.exam_id = ?
        """, [student_id, self.exam_id]).fetchone()
        
        if not student_info:
            return None
        
        # 获取错题记录
        errors = self.conn.execute("""
            SELECT 
                pr.question_id,
                pr.question_position,
                pr.error_category,
                eq.section as question_type,
                eq.points
            FROM practice_records pr
            JOIN exam_questions eq ON pr.exam_id = eq.exam_id AND pr.question_position = eq.question_number
            WHERE pr.student_id = ? 
                AND pr.exam_id = ?
                AND pr.is_correct = 0
            ORDER BY pr.question_position
        """, [student_id, self.exam_id]).fetchall()
        
        # 分析错误类型
        knowledge_errors = []
        careless_errors = []
        
        for error in errors:
            error_info = {
                'question_id': error[0],
                'position': error[1],
                'type': error[3],
                'points': error[4]
            }
            
            if error[2] == '知识性':
                knowledge_errors.append(error_info)
            elif error[2] == '粗心':
                careless_errors.append(error_info)
        
        return {
            'student_id': student_id,
            'name': student_info[0],
            'mbti': student_info[1],
            'total_score': student_info[2],
            'rank': student_info[3],
            'knowledge_errors': knowledge_errors,
            'careless_errors': careless_errors
        }
    
    def recommend_for_knowledge_errors(self, question_ids: List[str]) -> Dict:
        """针对知识性错误推荐练习"""
        if not question_ids:
            return {'weak_skills': [], 'recommended_questions': []}
        
        # 获取错题的题型
        placeholders = ','.join(['?'] * len(question_ids))
        error_questions = self.conn.execute(f"""
            SELECT DISTINCT eq.section
            FROM practice_records pr
            JOIN exam_questions eq ON pr.exam_id = eq.exam_id AND pr.question_position = eq.question_number
            WHERE pr.question_id IN ({placeholders})
        """, question_ids).fetchall()
        
        # 生成薄弱点
        weak_skills = []
        for (q_type,) in error_questions:
            if q_type == '单选题':
                weak_skills.append('基础概念理解')
            elif q_type == '多选题':
                weak_skills.append('综合分析能力')
            elif q_type == '填空题':
                weak_skills.append('计算能力')
            elif q_type == '解答题':
                weak_skills.append('解题思路')
        
        # 去重
        weak_skills = list(set(weak_skills))
        
        # 生成推荐题目
        recommended_questions = []
        for (q_type,) in error_questions:
            recommended_questions.append({
                'question_id': f'PRACTICE_{q_type}',
                'type': q_type,
                'points': 10 if q_type in ['单选题', '填空题'] else 20 if q_type == '多选题' else 50,
                'reason': f'巩固{q_type}的解题技巧'
            })
        
        return {
            'weak_skills': weak_skills,
            'recommended_questions': recommended_questions[:4]  # 最多推荐4道题
        }
    
    def analyze_class(self) -> List[Dict]:
        """分析全班学生情况"""
        # 获取所有学生
        students = self.conn.execute("""
            SELECT student_id FROM students 
            WHERE class_id = (SELECT class_id FROM exams WHERE exam_id = ?)
            ORDER BY student_id
        """, [self.exam_id]).fetchall()
        
        class_analysis = []
        
        for student_id, in students:
            # 分析每个学生
            student_analysis = self.analyze_student_errors(student_id)
            if not student_analysis:
                continue
            
            # 生成个人分析报告
            report = {
                'student_id': student_id,
                'name': student_analysis['name'],
                'score': student_analysis['total_score'],
                'rank': student_analysis['rank'],
                'mbti': student_analysis['mbti'],
                'analysis': {}
            }
            
            # 分析知识性错误
            if student_analysis['knowledge_errors']:
                knowledge_question_ids = [e['question_id'] for e in student_analysis['knowledge_errors']]
                recommendations = self.recommend_for_knowledge_errors(knowledge_question_ids)
                
                report['analysis']['knowledge'] = {
                    'error_count': len(student_analysis['knowledge_errors']),
                    'lost_points': sum(e['points'] for e in student_analysis['knowledge_errors']),
                    'weak_skills': recommendations['weak_skills'],
                    'recommendations': recommendations['recommended_questions'][:4]  # 最多4道题
                }
            
            # 分析粗心错误
            if student_analysis['careless_errors']:
                report['analysis']['careless'] = {
                    'error_count': len(student_analysis['careless_errors']),
                    'lost_points': sum(e['points'] for e in student_analysis['careless_errors']),
                    'positions': [e['position'] for e in student_analysis['careless_errors']],
                    'alert': '需要关注做题习惯和审题能力'
                }
            
            class_analysis.append(report)
        
        return class_analysis
    
    def generate_summary_report(self) -> Dict:
        """生成完整的分析报告"""
        print("=" * 60)
        print("考试分析报告")
        print("=" * 60)
        
        # 1. 考试概况
        overview = self.analyze_exam_overview()
        print(f"\n考试名称：{overview['exam_name']}")
        print(f"考试日期：{overview['exam_date']}")
        print(f"满分：{overview['full_score']}分")
        print(f"\n参考人数：{overview['total_students']}人")
        print(f"平均分：{overview['avg_score']}分")
        print(f"最高分：{overview['max_score']}分")
        print(f"最低分：{overview['min_score']}分")
        print(f"优秀率：{overview['excellent_rate']}%")
        print(f"及格率：{overview['pass_rate']}%")
        
        print("\n各题统计：")
        print(overview['question_stats'].to_string(index=False))
        
        # 2. 学生个人分析
        print("\n\n" + "=" * 60)
        print("学生个人分析")
        print("=" * 60)
        
        class_analysis = self.analyze_class()
        
        # 统计汇总
        knowledge_error_students = []
        careless_error_students = []
        
        for student in class_analysis:
            print(f"\n【{student['name']}】 学号：{student['student_id']} "
                  f"成绩：{student['score']}分 排名：第{student['rank']}名")
            
            if 'knowledge' in student['analysis']:
                knowledge_error_students.append(student['name'])
                ka = student['analysis']['knowledge']
                print(f"  知识性错误：{ka['error_count']}题，失分{ka['lost_points']}分")
                print(f"  薄弱知识点：{', '.join(ka['weak_skills'])}")
                print("  推荐练习：")
                for i, q in enumerate(ka['recommendations'], 1):
                    print(f"    {i}. {q['question_id']} ({q['type']},  - {q['reason']}")
            
            if 'careless' in student['analysis']:
                careless_error_students.append(student['name'])
                ca = student['analysis']['careless']
                print(f"  ⚠️ 粗心错误：{ca['error_count']}题，失分{ca['lost_points']}分")
                print(f"  错误位置：第{', '.join(map(str, ca['positions']))}题")
                print(f"  建议：{ca['alert']}")
        
        # 3. 总结与建议
        print("\n\n" + "=" * 60)
        print("总结与建议")
        print("=" * 60)
        
        print(f"\n需要加强知识点学习的学生（{len(knowledge_error_students)}人）：")
        print(f"  {', '.join(knowledge_error_students[:10])}" + 
              ("等" if len(knowledge_error_students) > 10 else ""))
        
        print(f"\n需要改善做题习惯的学生（{len(careless_error_students)}人）：")
        print(f"  {', '.join(careless_error_students[:10])}" + 
              ("等" if len(careless_error_students) > 10 else ""))
        
        # 返回结构化数据供助教agent使用
        return {
            'overview': overview,
            'class_analysis': class_analysis,
            'summary': {
                'knowledge_error_students': knowledge_error_students,
                'careless_error_students': careless_error_students,
                'key_findings': self._generate_key_findings(overview, class_analysis)
            }
        }
    
    def _generate_key_findings(self, overview: Dict, class_analysis: List[Dict]) -> List[str]:
        """生成关键发现"""
        findings = []
        
        # 整体表现
        if overview['avg_score'] < overview['full_score'] * 0.7:
            findings.append("班级整体成绩偏低，需要加强基础知识教学")
        
        # 题目分析F
        question_stats = overview['question_stats']
        low_accuracy = question_stats[question_stats['正确率'] < 0.5]
        if not low_accuracy.empty:
            findings.append(f"第{', '.join(map(str, low_accuracy['题号'].tolist()))}题正确率较低，需要重点讲解")
        
        # 错误类型分析
        total_students = len(class_analysis)
        careless_ratio = len([s for s in class_analysis if 'careless' in s['analysis']]) / total_students
        if careless_ratio > 0.3:
            findings.append(f"超过{int(careless_ratio*100)}%的学生存在粗心错误，建议加强审题训练")
        
        return findings
    
    def close(self):
        """关闭数据库连接"""
        self.conn.close()


# 使用示例
if __name__ == "__main__":
    # 创建分析系统
    analyzer = SimpleAnalysisSystem("school_management.db", "EXAM_2024_TEST_001")
    
    try:
        # 生成完整报告
        report = analyzer.generate_summary_report()
        
        # 报告数据可以传递给助教agent
        print("\n\n[数据已准备好传递给助教Agent生成正式报告]")
        
    finally:
        analyzer.close()

考试分析报告

考试名称：2024年秋季学期第10周周测
考试日期：2025-06-22
满分：100分

参考人数：30人
平均分：70.4分
最高分：100.0分
最低分：35.0分
优秀率：36.7%
及格率：60.0%

各题统计：
 题号  题型  分值      正确率  知识性错误数  粗心错误数
  1 单选题  10 0.900000       2      1
  2 单选题  10 0.800000       6      0
  3 多选题  20 0.500000       2     13
  4 填空题  10 0.666667      10      0
  5 解答题  50 0.500000       0     15


学生个人分析

【黄婷】 学号：2024001 成绩：98.0分 排名：第3名

【杨浩】 学号：2024002 成绩：55.0分 排名：第19名
  ⚠️ 粗心错误：2题，失分70分
  错误位置：第3, 5题
  建议：需要关注做题习惯和审题能力

【杨娟】 学号：2024003 成绩：90.0分 排名：第4名
  ⚠️ 粗心错误：1题，失分10分
  错误位置：第1题
  建议：需要关注做题习惯和审题能力

【马腾】 学号：2024004 成绩：55.0分 排名：第20名
  ⚠️ 粗心错误：2题，失分70分
  错误位置：第3, 5题
  建议：需要关注做题习惯和审题能力

【马丽】 学号：2024005 成绩：45.0分 排名：第26名
  知识性错误：1题，失分20分
  薄弱知识点：综合分析能力
  推荐练习：
    1. PRACTICE_多选题 (多选题,  - 巩固多选题的解题技巧
  ⚠️ 粗心错误：1题，失分50分
  错误位置：第5题
  建议：需要关注做题习惯和审题能力

【陈强】 学号：2024006 成绩：35.0分 排名：第28名
  知识性错误：2题，失分20分
  薄弱知识点：计算能力, 基础概念理解
  推荐练习：
    1. PRACTICE_单选题 (单选题,  - 巩固单选题的解题技巧
    2. PRACTICE_填空题 (填空题,  - 巩固填空题的解题技巧
  ⚠️ 粗心错误：2题，失分70分
  错误位置：第3, 5题
  建议：需要关注做题

In [2]:
print(report)

{'overview': {'exam_name': '2024年秋季学期第11周周测', 'exam_date': '2025-06-22', 'full_score': 100, 'total_students': 30, 'avg_score': 72.7, 'max_score': 100.0, 'min_score': 25.0, 'excellent_rate': 33.3, 'pass_rate': 73.3, 'fail_count': 8, 'question_stats':    题号   题型  分值       正确率  知识性错误数  粗心错误数
0   1  单选题  10  0.900000       2      1
1   2  单选题  10  0.700000       9      0
2   3  多选题  20  0.466667       3     13
3   4  填空题  10  0.766667       7      0
4   5  解答题  50  0.566667       0     13}, 'class_analysis': [{'student_id': '2024001', 'name': '黄波', 'score': 62.0, 'rank': 22, 'mbti': 'ESTP', 'analysis': {}}, {'student_id': '2024002', 'name': '胡俊', 'score': 55.0, 'rank': 23, 'mbti': 'INFP', 'analysis': {}}, {'student_id': '2024003', 'name': '徐勇', 'score': 90.0, 'rank': 6, 'mbti': 'INFJ', 'analysis': {}}, {'student_id': '2024004', 'name': '刘腾', 'score': 80.0, 'rank': 11, 'mbti': 'ISFJ', 'analysis': {}}, {'student_id': '2024005', 'name': '黄腾', 'score': 100.0, 'rank': 1, 'mbti': 'ENFJ', 'analys

# 作业批改多轮对话带反思功能实现

In [ ]:
import os, io
from pydantic import BaseModel
from typing import List, Optional, Literal
from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI

client = OpenAI()

# 定义结构化输出的数据模型
class QuestionGrading(BaseModel):
    your_answer: str
    student_answer: str
    studentis_correct_or_not: bool
    why_is_correct_or_not: str
    score_rate: float  # 0.0 到 1.0
    error_category: Optional[Literal["知识性", "粗心"]] = None  # 只能是"知识性"或"粗心"，或None
    comment: str

class Reflection(BaseModel):
    your_answer: str
    agree_or_disagree: bool
    why_agree_or_disagree: str


# Function to create a file with the Files API
def create_file(file_path):
  with open(file_path, "rb") as file_content:
    result = client.files.create(
        file=file_content,
        purpose="vision",
    )
    return result.id

# Getting the file ID
file_id = create_file("answered_questions_pool/q4_completion_answered_correctly.jpg")

response_1 = client.responses.parse(
    model="o4-mini",
    input=[{
        "role": "user",
        "content": [
            {"type": "input_text", "text": "返回每道题学生是否答对以及为什么"},
            {
                "type": "input_image",
                "file_id": file_id,
                "detail": "high",
            },
        ],
    }],
    text_format=QuestionGrading,
)

print("第一轮对话",response_1.output_parsed.your_answer)

response_2 = client.responses.parse(
    model="o4-mini",
    input=[{
        "role": "user",
        "content": [
            {"type": "input_text", "text": f"这道题有人说答案是{response_1.output_parsed.your_answer}，你觉得答案是什么？"},
            {
                "type": "input_image",
                "file_id": file_id,
                "detail": "high",
            },
        ],
    }],
    text_format=Reflection,
)

print("第二轮对话的反思结果",response_2.output_parsed.agree_or_disagree)
if not response_2.output_parsed.agree_or_disagree:
   print("请转人工")